In [ ]:
import os
os.chdir('../')
print(os.getcwd())

In [2]:
import pandas as pd
from PIL import Image
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [4]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 10
num_classes = 2
batch_size = 2
learning_rate = 0.0001

In [5]:
from src.dataset import single_task_dataset

In [6]:
df = pd.read_csv('input/data.csv')

In [7]:
df_train = df.loc[(df.Type=='train') & (df.Subject!=10)].reset_index(drop = True)
df_test = df.loc[(df.Type=='test' )& (df.Subject!=10)].reset_index(drop = True)
df_train.shape, df_test.shape

((2760, 5), (720, 5))

In [8]:
train_dataset = single_task_dataset(df_train)
test_dataset = single_task_dataset(df_test)

In [9]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [10]:
def conv_block(in_channels, out_channels):
    '''Convolution Block of 3x3 kernels + batch norm + maxpool of 2x2'''

    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 5, padding=2),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.conv1 = conv_block(1, 20)
        self.conv2 = conv_block(20, 40)
        self.conv3 = conv_block(40, 60)

        self.head = nn.Linear(9 * 4 * 60, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        # Features of shape (batch_size, 64)
        feat = x.reshape(x.size(0), -1)

        # Output
        out = self.head(feat)

        return out

In [11]:
model = ConvNet().to(device)

In [12]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        # print('shapes = ', outputs.shape, labels.shape)
        # print(outputs, labels)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/1380], Loss: 0.6316
Epoch [1/10], Step [200/1380], Loss: 1.1057
Epoch [1/10], Step [300/1380], Loss: 0.8966
Epoch [1/10], Step [400/1380], Loss: 0.4544
Epoch [1/10], Step [500/1380], Loss: 0.4107
Epoch [1/10], Step [600/1380], Loss: 0.3748
Epoch [1/10], Step [700/1380], Loss: 0.6129
Epoch [1/10], Step [800/1380], Loss: 1.1362
Epoch [1/10], Step [900/1380], Loss: 0.6692
Epoch [1/10], Step [1000/1380], Loss: 1.0942
Epoch [1/10], Step [1100/1380], Loss: 0.6401
Epoch [1/10], Step [1200/1380], Loss: 0.8300
Epoch [1/10], Step [1300/1380], Loss: 0.3906
Epoch [2/10], Step [100/1380], Loss: 0.6169
Epoch [2/10], Step [200/1380], Loss: 0.7066
Epoch [2/10], Step [300/1380], Loss: 0.6753
Epoch [2/10], Step [400/1380], Loss: 1.2899
Epoch [2/10], Step [500/1380], Loss: 0.3424
Epoch [2/10], Step [600/1380], Loss: 0.8945
Epoch [2/10], Step [700/1380], Loss: 1.3350
Epoch [2/10], Step [800/1380], Loss: 0.1776
Epoch [2/10], Step [900/1380], Loss: 0.6430
Epoch [2/10], Step [1000/138

In [14]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the test images: 72.08333333333333 %
